In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("/home/vmeshchaninov/DiffusionTextGeneration-cond-ca/")

In [3]:
from datasets import load_from_disk, Dataset

In [4]:
dt = load_from_disk("/home/vmeshchaninov/nlp_models/data/common_gen/valid")

In [5]:
dt

Dataset({
    features: ['concept_set_idx', 'concepts', 'target'],
    num_rows: 4018
})

In [6]:
dt.data

MemoryMappedTable
concept_set_idx: int32
concepts: list<item: string>
  child 0, item: string
target: string
----
concept_set_idx: [[0,0,0,0,1,...,269,270,270,270,270],[271,271,271,271,272,...,532,532,533,533,533],...,[752,753,753,753,753,...,988,988,988,988,988],[989,989,989,989,990,...,992,992,992,992,992]]
concepts: [[["field","look","stand"],["field","look","stand"],...,["sit","watch","ground"],["sit","watch","ground"]],[["book","sit","bed"],["book","sit","bed"],...,["score","field","goal","run"],["score","field","goal","run"]],...,[["tree","plant","ground","hole","dig"],["hit","player","run","base","ball"],...,["hold","pump","stand","car","nozzle"],["hold","pump","stand","car","nozzle"]],[["wheelbarrow","push","fall","sit","hill"],["wheelbarrow","push","fall","sit","hill"],...,["perform","wear","dance","stage","costume"],["perform","wear","dance","stage","costume"]]]
target: [["The player stood in the field looking at the batter.","The coach stands along the field, looking at the 

In [13]:
data = {}

In [14]:
for t in dt:
    id_ = t["concept_set_idx"]
    concepts = t["concepts"]
    target = t["target"]
    
    if id_ in data:
        data[id_]["targets"].append(target)
    else:
        data[id_] = {
            "concept_set_idx": id_,
            "concepts": concepts,
            "targets": [target],
        }

In [15]:
data[0]

{'concept_set_idx': 0,
 'concepts': ['field', 'look', 'stand'],
 'targets': ['The player stood in the field looking at the batter.',
  'The coach stands along the field, looking at the goalkeeper.',
  'I stood and looked across the field, peacefully.',
  'Someone stands, looking around the empty field.']}

In [17]:
d = Dataset.from_list(list(data.values()))

In [19]:
d[1]

{'concept_set_idx': 1,
 'concepts': ['kid', 'room', 'dance'],
 'targets': ['The silly kid loves to dance in her room.',
  'the dance kid  room is full of kids',
  'A kid is dancing in the room.',
  'A group of kids are dancing around a living room.']}

In [11]:
mult_ref = []
pred = []

In [16]:
for d in data.values():
    pred.append(d["targets"][0])
    mult_ref.append(d["targets"][1:])

In [17]:
from estimation_utils.evaluation import compute_common_gen_metrics

/home/vmeshchaninov/.conda/envs/fap2_env/lib/python3.9/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [25]:
from aac_metrics.functional import bleu

In [26]:
results = bleu(pred, mult_ref)

In [11]:
from data.dataset import CommonGenDatasetDDP

In [13]:
dt = next(CommonGenDatasetDDP(split="valid", base_path="/home/vmeshchaninov/nlp_models/data/common_gen").get_data())

Dataset tokenization (num_proc=30):   0%|          | 0/993 [00:00<?, ? examples/s]

In [14]:
dt

Dataset({
    features: ['concept_set_idx', 'target', 'mult_references', 'text_src', 'text_trg'],
    num_rows: 993
})